<a href="https://colab.research.google.com/github/schizyfos/Azure_DP_100/blob/master/nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 230, done.
remote: Total 230 (delta 0), reused 0 (delta 0), pack-reused 230
Receiving objects: 100% (230/230), 4.38 MiB | 16.07 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [0]:
import os
os.chdir("gpt-2")




In [4]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 42kB/s 
     |████████████████████████████████| 3.8MB 49.0MB/s 
     |████████████████████████████████| 512kB 50.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=02f07632432c009e86e1528c5b3a6aab75591300cf49600b63f5b2230b90a262
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uninstalling te

In [5]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 2.7MB/s 
     |████████████████████████████████| 604kB 10.5MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=366504684de0f3aa0f18f5ab72b7a0fdcdae30b1ff81fe18bc9f221aaf7956d9
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533186 sha256=0f6ccdc8c50b3f1f2e843671f174a81a41da2c5eb457d538fbdc77d47f564fac
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.31.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but

In [29]:
!python3 download_model.py 355M

Fetching checkpoint: 1.00kit [00:00, 1.16Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 54.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.03Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:30, 46.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 10.5Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 37.3Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 35.5Mit/s]                                                       


In [9]:
pip list | grep tensorflow

tensorflow               2.2.0          
tensorflow-addons        0.8.3          
tensorflow-datasets      2.1.0          
tensorflow-estimator     1.15.1         
tensorflow-gcs-config    2.1.8          
tensorflow-gpu           1.15.0         
tensorflow-hub           0.8.0          
tensorflow-metadata      0.22.0         
tensorflow-privacy       0.2.2          
tensorflow-probability   0.10.0         


In [0]:

!export PYTHONIOENCODING=UTF-8

In [0]:
os.chdir('src')

In [0]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [0]:
interact_model(
    '355M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)

INFO:tensorflow:Restoring parameters from /content/gpt-2/models/355M/model.ckpt
Model prompt >>> anton webern
======================================== SAMPLE 1 ========================================
coeson@mccrrbell.com

Art by Rachel McKerron, Adam Guilfoyle, Howard Hamberger, Reid Bruni

Cover by Adam Guilfoyle

Variant cover by Christine Choi

On sale SEPTEMBER 25 • 32 pg, FC, $2.99 US • RATED T

Retailers: This issue will ship with two covers. Please see the order form for details.

It's time for there to be a suicide amnesty on the Silver Surfer's world! But Thor isn't going to take to despair when Kurt Russell takes over! Maybe the world is finally being saved—Happy birthday! . . revealed!

Cosmic Joy, No Mercy, Cypher Magic, Lucky Darren and The Horror Killer are taking over the Marvel Cinematic Universe in "Birds of Prey," in which the Hawkeye free-form investigation calls on close allies Doc Manhattan, Infinity Monitor and Fury. But, reality is not much better on Earth. At l